# 1.0 Reranking file

## 1.1 Libray imports

In [5]:
import sys
import os

# Go up one directory to access /src
sys.path.append(os.path.abspath(".."))
import pandas as pd
import nest_asyncio
import asyncio

from datetime import datetime

### 1.2 Import files

In [6]:
from src.reranker import rerank_routes
from src.route_finder import find_routes
from src.crowd_predictor import load_model, predict_crowdedness
from src.summarizer import generate_trip_summary
from src.weather import get_current_weather
from src.user_input_planner import plan_trip
from src.alerts_dynamic import get_alerts_for_stop_async

nest_asyncio.apply()

## 2.1 Graph simulation of routes

In [7]:
graph = {
    "15L": ["A","B","C","D","E"],
    "0": ["F","C","G"],
    "1": ["H","I","J","E"],
}

## 2.2 Find routes

In [8]:
candidates = find_routes("B","E", graph)

## 2.3 Mock crowd, delay, and safety scores

In [9]:
for c in candidates:
    c["walk_time"] = 5
    c["delay_minutes"] = 4
    c["crowdedness_score"] = 30 # mock
    c["safety_score"] = 70 # mock

## 2.4 Turn into Data frame for reranking

In [10]:
df = pd.DataFrame(candidates)

## 3.0 Score and sort the results

In [11]:
reranked = rerank_routes(df)

## 4.0 Display

In [12]:
display(reranked)

,route_id,origin,destination,stops,bus_time,walk_time,delay_minutes,crowdedness_score,safety_score,total_score
0,15L,B,E,"[B, C, D, E]",12,5,4,30,70,18.5


In [13]:
model = load_model("models/crowd_model.pkl")

## 5.0 Assume 'df' contains route_id, hour_of_day, day_of_week

In [14]:
df["hour_of_day"] = 17
df["day_of_week"] = "Monday"

## 6.0 Predict crowdedness (returns 0==Low, 1==Medium, 2==High)

In [15]:
df["crowdedness_score"] = predict_crowdedness(model, df)

### 7.0 Convert back to readable labels

In [16]:
label_map = {0: "Low", 1: "Medium", 2: "High"}
df["crowdedness_label"] = df["crowdedness_score"].map(label_map)

## 8.0 Display top candidates

In [17]:
display(df[["route_id", "origin", "destination", "crowdedness_score", "crowdedness_label"]])

,route_id,origin,destination,crowdedness_score,crowdedness_label
0,15L,B,E,1,Medium


## 9.0 Mock trip planner

In [18]:
trip = {
    "walk_time": 5,
    "route_id": "15L",
    "stop_name_start": "Blake & 14th",
    "stop_name_end": "Decatur Station",
    "departure_time": "7:14 PM",
    "arrival_time": "7:38 PM",
    "destination": "840 Irving Street"
}

## 10.0 Summarize the trip

In [19]:
summary = generate_trip_summary(trip)
print(summary)

Walk 5 minutes to Blake & 14th. Take the 15L bus at 7:14 PM. Get off at Decatur Station. You’ll arrive at 840 Irving Street by 7:38 PM.


## 11. User defined trip details

In [20]:
user_start = "B"
user_end = "E"
user_time = datetime.strptime("2025-04-16 17:00", "%Y-%m-%d %H:%M")

## 11.1 Verify weather

In [21]:
weather = get_current_weather(city="Denver")
print("Current Weather:", weather)

Current Weather: Clear


In [23]:
alerts = asyncio.run(get_alerts_for_stop_async("Florida Station"))
for alert in alerts:
    print("⚠️", alert)


Task exception was never retrieved
future: <Task finished name='Task-4' coro=<Connection.run() done, defined at c:\Users\Christian\Desktop\TripGuardian\venv\Lib\site-packages\playwright\_impl\_connection.py:272> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\Christian\anaconda3\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\Christian\Desktop\TripGuardian\venv\Lib\site-packages\playwright\_impl\_connection.py", line 279, in run
    await self._transport.connect()
  File "c:\Users\Christian\Desktop\TripGuardian\venv\Lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "c:\Users\Christian\Desktop\TripGuardian\venv\Lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Christian\a

NotImplementedError: 

## 11.2 Generate reranked trips

In [ ]:
results, msg = plan_trip(
    user_start="B",
    user_end="E",
    departure_time=user_time,
    route_graph=graph,
    weather="Rain",
    user_is_disabled=True
)
print(msg)


## 13.0 Show top trip options with summaries

In [ ]:
results[[
    "route_id", 
    "crowdedness_label", 
    "safety_score",
    "weather", 
    "total_score", 
    "trip_summary",
    "alert_summary"
]]